In [1]:
# ToDo:
#  - test invalid imports
#  - uv on jupyterhub? isolated environment? use our .venv?
#  - timeouts: https://stackoverflow.com/questions/2281850/timeout-function-if-it-takes-too-long-to-finish
    # https://stackoverflow.com/questions/45426713/how-to-gracefully-timeout-with-asyncio
#      - also consider sandboxing it
#      - flag execs?
#  - kernel crashes; keep a log
#  - sort users alphabetically (done by inspect.getmembers)
#  - convert to classes
#  - utility for students to check before submitting?
#  - decorate functions with generic error handler to capture and report exceptions

In [2]:
import student_code
import key

# Don't forget to add everything to the venv
from types import ModuleType
from typing import Callable

import concurrent.futures
import threading

from traceback import format_exception

import pandas as pd
import inspect

KEY_FUNC = 'func'
KEY_RETURN = 'return'
KEY_QUESTION = 'q'
KEY_SKIP = '_'

In [3]:
args = {
    'q1': (3,),
    'q2': (4,5),
    'q3': ()
}

In [4]:
# Create the key
pkey = {}
for name, obj in inspect.getmembers(key, predicate=inspect.isfunction):
    #if inspect.isfunction(obj) and name.startswith('q'):
    if name.startswith(KEY_QUESTION):
        pkey[name] = {}
        pkey[name][KEY_FUNC] = obj
        pkey[name][KEY_RETURN] = obj(*args[name])
#pkey

In [5]:
# Get the individual users
students = {}
for name, obj in inspect.getmembers(student_code, predicate=inspect.ismodule):
    #if inspect.ismodule(obj) and not name.startswith('_'):
    if not name.startswith(KEY_SKIP):
        students[name] = obj
students

{'user1': <module 'student_code.user1' from '/Users/rdtls/Desktop/pytest/code_tester/student_code/user1.py'>,
 'user2': <module 'student_code.user2' from '/Users/rdtls/Desktop/pytest/code_tester/student_code/user2.py'>,
 'user3': <module 'student_code.user3' from '/Users/rdtls/Desktop/pytest/code_tester/student_code/user3.py'>}

In [6]:
# For output:
grade_sheet = pd.DataFrame(index=list(students.keys()), columns=list(pkey.keys()))

def user_output(q_nums: list[str]) -> pd.DataFrame:
    columns = ['Arguments', 'Output', 'Expected']
    return pd.DataFrame(index=q_nums, columns=columns)

In [44]:
def function_handler(func: Callable) -> Callable:
    def wrapper(*args, **kwargs):
        try:
            results = func(*args, **kwargs)
        except Exception as e:
            results = e
        return results
    return wrapper

In [66]:
def fun_handler(timeout_secs: float = None) -> Callable:
    class TimeoutError(Exception):
        pass
    def timeout(func_name: str) -> None:
        raise TimeoutError(f'{func_name} timed out after {timeout_secs} second(s).')
        return
    def decorator(func: Callable) -> Callable:
        def wrapper(*args, **kwargs):
            if timeout_secs is not None:
                timer = threading.Timer(timeout_secs, timeout, args=[func.__name__])
                timer.start()
            try:
                results = func(*args, **kwargs)
            except Exception as e:
                results = e
            finally:
                if timeout_secs is not None:
                    timer.cancel()
            return results
        return wrapper
    return decorator

In [25]:
# For each user:
#   -Create their output sheet
#   -Write to the log
#   -Search for function, run and log results

In [33]:
def grade_student(code: ModuleType):
    # Initialize their output
    
    qcode = {}
    for name, obj in inspect.getmembers(code, predicate=inspect.isfunction):
        if name.startswith(KEY_QUESTION):
            qcode[name] = {}
            qcode[name][KEY_FUNC] = obj
    print(qcode)

    # For each question in the key...
    for q in pkey.keys():
        # Check if the question exists in the student code
        if q in qcode.keys():
            print(f'{q} found in student code')
            # Run the function, store the output, update the score sheet
        else:
            print(f'{q} not found in student code')
    return

In [34]:
grade_student(students['user3'])

{'q1': {'func': <function q1 at 0x0000014DFEEC2F20>}, 'q2': {'func': <function q2 at 0x0000014DFEEC2DE0>}}
q1 found in student code
q2 found in student code
q3 not found in student code


In [93]:
import time

#@fun_handler(1)

def fun():
    #x = 1/0
    time.sleep(20)
    return 3

In [94]:
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = [executor.submit(fun)]
    done, not_done = concurrent.futures.wait(futures, timeout=1)

print('d')
for fs in done:
    print(fs.result())
print('nd')
for fs in not_done:
    print(fs.result())

d
nd
3


3


In [49]:
users = [item for item in dir(user_code) if not item.startswith('_')]
users

['user1', 'user2', 'user3']

In [50]:
questions = [item for item in dir(key) if item.startswith('q')]
questions

['q1', 'q2']

In [47]:
for key, value in pkey.items():
    print(key, value)

q1 <function q1 at 0x0000021DFA6F1D00>
q2 <function q2 at 0x0000021DFA6F2020>


---

In [14]:
import ast
with open('key.py', 'r') as f:
    node = ast.parse(f.read())

fun = node.body[1]

In [15]:
dir(fun)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_field_types',
 '_fields',
 'args',
 'body',
 'col_offset',
 'decorator_list',
 'end_col_offset',
 'end_lineno',
 'lineno',
 'name',
 'returns',
 'type_comment',
 'type_params']